In [62]:
import json
import plotly.express as px
import plotly.graph_objects as go
import os
import pandas as pd
from statistics import mean, median, stdev, variance
import qgrid

In [66]:
# folder paths
results_folder = 'results_tiastower/2020_05_09/'
gurobi_folder = 'Gurobi/'
ortools_folder = 'OrTools/'
gurobi_cold_folder = 'GurobiCold/'
extension1 = 'ext1/'
extension2 = 'ext2/'
extension3 = 'ext3/'
extension4 = 'ext4/'

# problem names
# p = set(f_name for f_name in gurobi_ext4_files)
p = set()

# solver 
solvers = {'or-tools':'OrTools/', 'gurobi-warm':'Gurobi/', 'gurobi-cold':'GurobiCold/'}

# extension
extensions = [1, 2, 3, 4]

for solver in solvers:
    solverFolder = solvers[solver]
    for extension in extensions:
        folder_path_extension = results_folder + solverFolder + f"ext{extension}/"
        if os.path.exists(folder_path_extension):
            files = set(f for f in os.listdir(folder_path_extension) if  f.endswith('.json'))
            #print(files)
            [p.add(f) for f in files]

# folder paths
f_paths = {}

for s in solvers:
    solver = solvers[s]
    f_paths[s] = {}
    for ext in extensions:
        f_paths[s][ext] = {}
        for problem in p:
            f_path = results_folder + solver + f"ext{ext}/" + problem
            if os.path.isfile(f_path):
                f_paths[s][ext][problem] = f_path

data = {'p':  [],
        'solver': [],
        'ext' : [],
        'clauses' : [],
        'steps': [],
        'total time [s]': [],
        '% hs [s]' : [],
        '% sat [s]' : [],
        '% grow [s]':[]
        }

for solver in f_paths:
    for ext in f_paths[solver]:
        for problem in f_paths[solver][ext]:
            f_path = f_paths[solver][ext][problem]

            # data parameters
            data['p'].append(problem)
            data['solver'].append(solver)
            data['ext'].append(ext)
            with open(f_path) as f:
                parsed_json = json.load(f)
            
            # data results
            data['clauses'].append(parsed_json['clauses'])
            data['steps'].append(parsed_json['steps'])
            tot_time = sum(parsed_json['t_hitting_set']) +sum(parsed_json['t_sat_check'])  + sum(parsed_json['t_grow'])
            data['total time [s]'].append(tot_time)
            data['% hs [s]'].append(round(100*sum(parsed_json['t_hitting_set'])/tot_time, 2))
            data['% sat [s]'].append(round(100*sum(parsed_json['t_sat_check'])/tot_time,2))
            data['% grow [s]'].append(round(100*sum(parsed_json['t_grow'])/tot_time, 2))
df = pd.DataFrame (data, columns = [column for column in data])
df = df.sort_values(["clauses", "steps", "total time [s]"], ascending = (False, True, True))
#df.to_excel("results_tiastower/2020_05_08.xlsx") 
#qgrid_widget = qgrid.show_grid(df, show_toolbar=True)
# qgrid_widget

# Results
## RQ1: for or-tools as HS solver, extension 1 vs 2 ia ib ic vs 3

- For OR-tools as optimal hittingset solver, we can see that in all the cases, **extension3** performs better than **extension 2**. 
- **Extension 1** provides results in a considerably longer time even for small instances...

**Conclusion RQ1**

Best extension = `extension 3`

In [1]:
# filter columns 
selected_columns = ['p','solver', 'ext', 'clauses', 'steps', 'total time [s]']

# data filter
df_ortools_ext123 = df[selected_columns][(df.ext >= 2) & (df.ext <= 3) & (df.solver == 'or-tools')]

# display table
df_ortools_ext123
#qgrid_widget = qgrid.show_grid(df_ortools_ext123, show_toolbar=True)
#qgrid_widget

NameError: name 'df' is not defined

## RQ2: for the best of RQ1, compare or-tools vs gurobi

We compare the results of `extension 3` for the 3 solvers : 

- `Or-Tools`: Or-tools optimal hitting set solver
- `Gurobi Warm Start`: Model is built on all variables and for every new Minimum Correction Set found, we had a new constraint corresponding to a new set in the collection of sets to hit.
- `Gurobi Cold Start`: The full model is rebuilt for every optimal hitting set call

**Results are sorted by decreasing number of clauses, Increasing problem name, total solving time**

- We can see that for `extension 3` the total solving time `total time [s]` is always the smallest with gurobi using a warm start. 
- For cold starts, or-tools performs better than gurobi.

**Conclusion RQ1**

Best combo = `gurobi warm start + extension 3`

In [52]:
# filter columns 
selected_columns = ['p','solver', 'ext', 'clauses', 'steps', 'total time [s]']

# data filter
df_solver_ext3 = df[selected_columns][(df.ext == 3)]
df_solver_ext3 = df_solver_ext3[(df_solver_ext3['solver'].isin(['or-tools', 'gurobi-warm', 'gurobi-cold']))]
df_solver_ext3 = df_solver_ext3.sort_values(
    ["clauses", "p", "total time [s]"], 
    ascending = (False, True, True))

# display marked-up table
qgrid_widget = qgrid.show_grid(df_solver_ext3, show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## RQ3: for the best of RQ2, compare  with maxsat

We compare the results :
- `Gurobi warm start + extension3` vs 
- `Gurobi warm start + Max Sat` 

We see that for the medium instances, extension 3 takes too much time (> 1 day).

1. bf0432-007
2. zebra_v155_c1135

For smaller instances, we see that the MaxSat solution takes :
- less steps 
- sometimes slower on smaller instances than the greedy approach of extension 3.

**Conclusion**

Overall the maxsat solution implemented by https://pysathq.github.io/docs/html/api/examples/rc2.html:

    Alexey Ignatiev, António Morgado, Joao Marques-Silva. RC2: An Efficient MaxSAT Solver. MaxSAT Evaluation 2018. JSAT 11. 2019. pp. 53-64

shows very good results and performs better on medium instances.

In [65]:
# filter columns 
selected_columns = ['p','solver', 'ext', 'clauses', 'steps', 'total time [s]']

# data filter
df_warm_start = df[selected_columns]
#print(df_warm_start)
df_warm_start = df_warm_start[df_warm_start.solver == 'gurobi-warm']
df_warm_start = df_warm_start[df_warm_start.ext >= 3]
df_warm_start = df_warm_start.sort_values(
    ["clauses", "p", "total time [s]"], 
    ascending = (False, True, True))

df_warm_start

,p,solver,ext,clauses,steps,total time [s]
34,bf0432-007.json,gurobi-warm,4,3667,1274,111.835520
37,zebra_v155_c1135.json,gurobi-warm,4,1160,544,136.551506
27,par8-1-c.json,gurobi-warm,3,250,2,0.002234
33,par8-1-c.json,gurobi-warm,4,250,2,0.016101
26,dubois22.json,gurobi-warm,3,176,193,0.302204
32,dubois22.json,gurobi-warm,4,176,176,0.461740
25,dubois21.json,gurobi-warm,3,168,194,0.313079
31,dubois21.json,gurobi-warm,4,168,168,0.442617
30,aim-100-1_6-no-1.json,gurobi-warm,4,160,47,0.104584
24,aim-100-1_6-no-1.json,gurobi-warm,3,160,180,1.661584


In [ ]:
for problem in gurobi_ext4_files:
    ext = 4
    fig = go.Figure()
    for solver in ['or-tools','gurobi-warm', 'gurobi-cold']:
        if not problem in f_paths[solver][ext]:
            continue
        f_path = f_paths[solver][ext][problem]
        with open(f_path) as f:
            parsed_json = json.load(f)
        # getting the data
        t_hitting_set = parsed_json['t_hitting_set']
        t_sat_check = parsed_json['t_sat_check']
        t_grow = parsed_json['t_grow']
        y_steps = [i for i in range(1,parsed_json['steps']+2)]
        y_steps_grow = [i for i in range(1,parsed_json['steps']+1)]
        fig.add_trace(go.Scatter(
                    y=t_hitting_set, 
                    x=y_steps,
                    mode='lines',
                    name=f'optimal hitting set {solver}'))
    # Edit the layout
    fig.update_layout(title=f'{problem} extension {ext}- Optimal Hitting set exec time vs Solver',
                       xaxis_title='Step',
                       yaxis_title='Execution time [s]')
    fig.show()
